In [1]:
import os
import subprocess

import pandas as pd

from google.cloud import bigquery
import google.auth.transport.requests
import pandas_gbq

/home/ext_meehl_joshua_mayo_edu/.conda/envs/pre-phd-genomics/lib/python3.10/site-packages/google/api_core/_python_version_support.py:266: FutureWarning: You are using a Python version (3.10.19) which Google will stop supporting in new releases of google.api_core once it reaches its end of life (2026-10-04). Please upgrade to the latest Python version, or at least Python 3.11, to continue receiving updates for google.api_core past that date.
  warnings.warn(message, FutureWarning)
/home/ext_meehl_joshua_mayo_edu/.conda/envs/pre-phd-genomics/lib/python3.10/site-packages/google/cloud/bigquery_storage_v1/__init__.py:19: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


# Connect to CGP.

- If you need to login, run `gcloud auth application-default login --no-launch-browser`

In [23]:
creds, project = google.auth.default()
bq_client = bigquery.Client(project=project, credentials=creds)

In [82]:
job = bq_client.query("SELECT * FROM `aif-usr-p-cbr-genai-1524.EDA.tapestry_inference_only`")
df = job.to_dataframe()
df.shape

(97373, 170)

In [83]:
cols = [
    'tap_kitid',
    
    'inferred_sex', 
    'sex_at_birth', 

    'ethnicity',
    'is_hispanic', 
    'race', 
    'gnomad_race', 
    
    'ehr_birthdate', 
    'age_at_sequence',  
        
    'tap_state_enrolled', 
    'tap_state_current', 
    'currently_consented', 
    
    'vcf_format_version',
    'vcf_genome_version',
    'vcf_assay_version',
    
    'has_ra',
    'has_ibd', 
    'has_ibd_crohns', 
    'has_ibd_uc', 
    'has_fh', 
    'has_prediabetes',
    'has_diabetes1', 
    'has_diabetes2', 
    'has_breast_cancer',
    'has_prostate_cancer', 
    'has_high_risk_prostate_cancer',
    'prostate_risk_score',
    
    'max_ldl',
    'height_cm',
    'weight_kg', 
    'bmi', 
    
    'education_level',
    'employment_status',
    'alcohol_oz_per_wk', 
    'has_alcohol_oz_per_wk', 
    'alcohol_oz_per_wk_mean',
    'smoking_years', 
    'smoking_packyears', 
    'smoking_status',

    'ALCOHOL_ABUSE', 
    'ALCOHOL_USE', 
    'ANEMIA', 
    'ANXIETY', 
    'ASTHMA',
    'BIPOLAR', 
    'CANCER', 
    'CARDIAC_ARRHYTHMIA', 
    'CEREBROVASCULAR_DISEASE',
    'CHRONIC_BACK_PAIN', 
    'CONGESTIVE_HEART_FAILURE', 
    'COPD',
    'CORONARY_ARTERY_DISEASE', 
    'DEMENTIA', 
    'DEPRESSION', 
    'DIABETES',
    'DIVERTICULAR_DISEASE', 
    'DRUG_ABUSE', 
    'FIBROMYALGIA', 
    'GERD', 
    'GOUT',
    'GYNECOLOGIC_DISORDER', 
    'HEADACHES', 
    'HEARING_LOSS', 
    'HYPERLIPIDEMIA',
    'HYPERTENSION',
	'HYPERTHYROIDISM',
	'INFLAMMATORY_SKIN_DISEASE',
    'INTERSTITIAL_LUNG_DISEASE',
	'IRRITABLE_BOWEL_DISEASE',
	'LIVER_DISEASE',
    'NEUROPATHY',
	'OBESITY',
	'OSTEOARTHRITIS',
	'OSTEOPOROSIS',
    'PARKINSONS_DISEASE',
	'PULMONARY_CIRCULATION_DISORDER',
    'PEPTIC_ULCER_DISEASE',
	'PERIPHERAL_VASCULAR_DISEASE',
    'PROSTATE_HYPERPLASIA',
	'PTSD',
	'RENAL_DISEASE',
    'SEVERE_VISION_REDUCTION',
	'SLEEP_DISORDER',
    'URINARY_INCONTINENCE',
    'VALVULAR_HEART_DISEASE',

]

In [84]:
mask = df['tap_kitid'] != ' '
df_tapestry = df.loc[mask, cols].copy()
df_tapestry.shape

(97373, 85)

In [85]:
df_tapestry.head()
cols = {'tap_kitid': '#IID'}
df_tapestry.rename(columns=cols, inplace=True)

df_tapestry.columns = df_tapestry.columns.str.upper()
df_tapestry = df_tapestry.sort_values('#IID').reset_index(drop=True)
df_tapestry.head()

,#IID,INFERRED_SEX,SEX_AT_BIRTH,ETHNICITY,IS_HISPANIC,RACE,GNOMAD_RACE,EHR_BIRTHDATE,AGE_AT_SEQUENCE,TAP_STATE_ENROLLED,...,PULMONARY_CIRCULATION_DISORDER,PEPTIC_ULCER_DISEASE,PERIPHERAL_VASCULAR_DISEASE,PROSTATE_HYPERPLASIA,PTSD,RENAL_DISEASE,SEVERE_VISION_REDUCTION,SLEEP_DISORDER,URINARY_INCONTINENCE,VALVULAR_HEART_DISEASE
0,AAGWLAE326,F,F,Not Hispanic or Latino,False,White,European (Non-Finnish),1973-01-10,48,FL,...,None,None,None,None,None,None,None,None,None,None
1,AAIWLAE717,F,F,Not Hispanic or Latino,False,White,European (Non-Finnish),1956-04-16,65,FL,...,None,None,None,None,None,None,None,None,None,None
2,AAIZAAR717,F,F,Not Hispanic or Latino,False,White,European (Non-Finnish),1950-10-08,71,AR,...,None,None,None,None,None,None,None,1,None,None
3,AAIZKAH036,F,F,Hispanic or Latino,True,White,European (Non-Finnish),1976-09-03,45,TX,...,None,None,None,None,None,None,None,None,None,None
4,AAJWLAE907,F,F,Not Hispanic or Latino,False,White,European (Non-Finnish),1980-08-26,41,IA,...,None,None,None,None,None,None,None,None,None,None


In [90]:
path = '/home/ext_meehl_joshua_mayo_edu/pre-phd-genomics/02_genomics_domain/data/plink/tapestry/genome_wide/genome_qc.psam'
df_psam = pd.read_csv(path, delim_whitespace=True)
cols = {'SEX': 'INFERRED_SEX_2'}
df_psam.rename(columns=cols, inplace=True)
df_psam.head()

/var/tmp/ipykernel_3567214/2950657449.py:2: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df_psam = pd.read_csv(path, delim_whitespace=True)


,#IID,INFERRED_SEX_2
0,AAGWLAE326,1
1,AAIWLAE717,1
2,AAIZAAR717,2
3,AAIZKAH036,2
4,AAJWLAE907,1


In [91]:
df_merge = pd.merge(df_psam, df_tapestry, on='#IID', how='inner').copy()
df_merge.insert(1, 'SEX', df_merge['SEX_AT_BIRTH'])
df_merge.head()

,#IID,SEX,INFERRED_SEX_2,INFERRED_SEX,SEX_AT_BIRTH,ETHNICITY,IS_HISPANIC,RACE,GNOMAD_RACE,EHR_BIRTHDATE,...,PULMONARY_CIRCULATION_DISORDER,PEPTIC_ULCER_DISEASE,PERIPHERAL_VASCULAR_DISEASE,PROSTATE_HYPERPLASIA,PTSD,RENAL_DISEASE,SEVERE_VISION_REDUCTION,SLEEP_DISORDER,URINARY_INCONTINENCE,VALVULAR_HEART_DISEASE
0,AAGWLAE326,F,1,F,F,Not Hispanic or Latino,False,White,European (Non-Finnish),1973-01-10,...,None,None,None,None,None,None,None,None,None,None
1,AAIWLAE717,F,1,F,F,Not Hispanic or Latino,False,White,European (Non-Finnish),1956-04-16,...,None,None,None,None,None,None,None,None,None,None
2,AAIZAAR717,F,2,F,F,Not Hispanic or Latino,False,White,European (Non-Finnish),1950-10-08,...,None,None,None,None,None,None,None,1,None,None
3,AAIZKAH036,F,2,F,F,Hispanic or Latino,True,White,European (Non-Finnish),1976-09-03,...,None,None,None,None,None,None,None,None,None,None
4,AAJWLAE907,F,1,F,F,Not Hispanic or Latino,False,White,European (Non-Finnish),1980-08-26,...,None,None,None,None,None,None,None,None,None,None


In [94]:
path = '/home/ext_meehl_joshua_mayo_edu/pre-phd-genomics/02_genomics_domain/data/plink/tapestry/genome_wide/metadata.csv'
df_merge.to_csv(path, sep=',', index=False)

path = '/home/ext_meehl_joshua_mayo_edu/pre-phd-genomics/02_genomics_domain/data/plink/tapestry/genome_wide/metadata.psam'
df_merge.to_csv(path, sep='\t', index=False)